# UFO Sightings Redux

by **Mazza Luca**, **Giada Galdiolo** and **Vasco Silva Pereira**

## Dataset

[UFO Sightings Redux *by National UFO Reporting Center*](https://github.com/rfordatascience/tidytuesday/blob/main/data/2023/2023-06-20/readme.md)



In [ ]:
# Load Dataset

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as ps
import string
from collections import Counter
from plotly.subplots import make_subplots
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')

df = pd.read_csv('./data/ufo_sightings.csv')
df_dayparts = pd.read_csv('./data/day_parts_map.csv')
df_places = pd.read_csv('./data/places.csv')
population_df = pd.read_csv('./data/states_population.csv')

df['id'] = range(1, len(df) + 1)
df.set_index('id', inplace=True)

df['posted_date'] = pd.to_datetime(df['posted_date'], format='%Y-%m-%d', errors='coerce')
df['reported_date_time'] = pd.to_datetime(df['reported_date_time'], errors='coerce')
df['reported_date_time_utc'] = pd.to_datetime(df['reported_date_time_utc'], errors='coerce')
df['duration_seconds'] = pd.to_timedelta(df['duration_seconds'], unit='s', errors='coerce')

In [ ]:
df.info()

In [ ]:
df.head()

# Setup

In [ ]:
def bump_fonts(fig, base=20):
    """Aumenta i font di titoli, assi e legenda."""
    fig.update_layout(
        font=dict(size=base),
        title_font=dict(size=base + 2),
        legend_font=dict(size=base),

        coloraxis_colorbar=dict(
            title_font=dict(size=base, color='black'),
            tickfont=dict(size=base - 2, color='black'),
        )
    ).update_xaxes(
        title_font=dict(size=base, color='black'),
        tickfont=dict(size=base - 2, color='black'),
        gridcolor='gray'  # opzionale: colore griglia
    ).update_yaxes(
        title_font=dict(size=base, color='black'),
        tickfont=dict(size=base - 2, color='black'),
        gridcolor='gray'  # opzionale: colore griglia
    )
    return fig

## Rising Trend in Sightings Begins in 1993, with a Peak in 2014 

This graph shows the number of UFO sightings per year.

It is clear that between the *1930s* and the *1990s*, where means of communication were not advanced yet, less reports were turned in.
In the midst of the *1990s* it is noticeable a drastic increase of reports, probably due to de advent of the Internet, which made possible such information to spread.
In fact, during those years, TV-Shows like **X-Files**, movies like **Independence Day** and **Men in Black** popularized the concept and the creation of discussion forums about the
topic spread. The proliferation of hand-held cameras and camcorders also allowed more people to document and share sightings.

There is then a peak near the midst of the *2010s*, which is probably due to the near-universal adoption of **smartphones**, which are equipped with cameras, making the
documentation and report even easier on social media.
In this years it is also notable a **trend** regarding paranormal topics, including UFOs, on social media like *YouTube*.

In [ ]:
#Sightings per Year

df_tmp = df.groupby(df['reported_date_time'].dt.year).size()
fig = px.line(data_frame=df_tmp) \
    .update_layout(
    xaxis_title="Reported Year",
    yaxis_title="Number of sightings",
    showlegend=False,
    height=800
) \
    .add_vline(x=2014, line_color='red') \
    .add_vline(x=1993, line_color='red')

bump_fonts(fig)
fig.show()

## The most frequent sightings last between 1 and 15 minutes

In [ ]:
bins = [pd.Timedelta(seconds=0),
        pd.Timedelta(seconds=10),
        pd.Timedelta(seconds=30),
        pd.Timedelta(seconds=60),
        pd.Timedelta(minutes=5),
        pd.Timedelta(minutes=15),
        pd.Timedelta(minutes=30),
        pd.Timedelta(minutes=60),
        pd.Timedelta(hours=3)]
labels = [
    "0-10 sec",
    "0-30 sec",
    "30-60 sec",
    "1-5 mins",
    "5-15 mins",
    "15-30 mins",
    "30-60 mins",
    "Over 1 hour"
]
category_order = {
    "duration_category": [
        "0-10 sec",
        "0-30 sec",
        "30-60 sec",
        "1-5 mins",
        "5-15 mins",
        "15-30 mins",
        "30-60 mins",
        "Over 1 hour"
    ]
}
df["duration_category"] = pd.cut(df["duration_seconds"], bins=bins, labels=labels, right=False)

fig = px.histogram(
    data_frame=df,
    x='duration_category',
    category_orders=category_order,
    height=800
).update_layout(
    xaxis_title="Duration",
    yaxis_title="Number of sightings",
    showlegend=False
)

bump_fonts(fig)
fig.show()


# Northwestern U.S. Shows a Higher Concentration of Sightings per Million People 

This graph shows the concentration of *Sighting reports* over the various States of the United States of America, where all sightings have been collected.

It is clear that states residing in the **western** part of the US have generally more sightings over the ones residing in the eastern parts, with some exception.

This is probably due to the presence of rural areas in this part of the country, like Nevada, New Mexico and Arizona, which have vast areas with low population density,
providing less light pollution and therefore better visibility of the night sky.
In this area it is notable to find deserts, mountains and expansive skies, ideal for the observation of this kind of event.

In this area (especially Nevada) there are numerous military bases, testing sites and aerospace facilities (Edwards Air Force Base, Area 51). This could lead the sighting of some
experimental aircraft to be mistaken for an UFO.

In [ ]:
state_counts = df['state'].value_counts().reset_index()
state_counts.columns = ['state', 'sightings']

merged_df = pd.merge(state_counts, population_df, on='state')
merged_df['sightings_per_million'] = (merged_df['sightings'] / merged_df['population']) * 1_000_000

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Raw Sightings Count", "Sightings per 1M Population"),
    specs=[[{"type": "choropleth"}, {"type": "choropleth"}]]
)

choropleth_raw = go.Choropleth(
    locations=merged_df['state'],
    locationmode='USA-states',
    z=merged_df['sightings'],
    colorscale='Viridis',
    colorbar_title="Sightings",
    zmin=merged_df['sightings'].min(),
    zmax=merged_df['sightings'].max(),
    name="Raw Sightings",
    showscale=False
)

choropleth_norm = go.Choropleth(
    locations=merged_df['state'],
    locationmode='USA-states',
    z=merged_df['sightings_per_million'],
    colorscale='Viridis',
    colorbar_title="Sightings per 1M",
    zmin=merged_df['sightings_per_million'].min(),
    zmax=merged_df['sightings_per_million'].max(),
    name="Per 1M Population"
)

fig.add_trace(choropleth_raw, row=1, col=1)
fig.add_trace(choropleth_norm, row=1, col=2)

fig.update_layout(
    height=800,
    title_text="UFO Sightings Distribution: Raw vs Per Million Population",
    geo=dict(scope='usa'),
    geo2=dict(scope='usa'),
)

bump_fonts(fig)

fig.show()

# Bright Lights in the Sky: Most Common Theme

In [ ]:
summaries = df['summary'].dropna()

stop_words = set(stopwords.words('english'))
custom_stopwords = {"ufo", "sighting", "flying", "object", "seen", "saw", "like"}
stop_words.update(custom_stopwords)

lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation + string.digits))
    words = word_tokenize(text)
    words = [
        lemmatizer.lemmatize(word)
        for word in words
        if word not in stop_words and len(word) > 2
    ]
    return words

word_in_summaries = Counter()

all_words = []
for summary in summaries:
    words = preprocess_text(summary)
    word_in_summaries.update(words)

word_freq_df = pd.DataFrame(word_in_summaries.items(), columns=['word', 'count']).sort_values(by='count',
                                                                                        ascending=False).head(20)
fig = px.bar(
    word_freq_df,
    y='word',
    x='count',
    labels={'word': 'Word', 'count': 'Frequency'},
    text='count',
    orientation='h',
    height=800
).update_layout(yaxis=dict(autorange="reversed"))

bump_fonts(fig)
fig.show()

# Sightings Peak at Dawn During Summer Months

In [ ]:
df['month'] = df['reported_date_time'].dt.month
df['hour'] = df['reported_date_time'].dt.hour

month_map = {
    1: "Jan", 2: "Feb", 3: "Mar", 4: "Apr",
    5: "May", 6: "Jun", 7: "Jul", 8: "Aug",
    9: "Sep", 10: "Oct", 11: "Nov", 12: "Dec"
}
df["month_name"] = df["month"].map(month_map)
month_order = ["Jan", "Feb", "Mar", "Apr", "May", "Jun",
               "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
df["hour_str"] = df["hour"].apply(lambda h: f"{h:02d}:00")

heatmap_data = df.groupby(['hour_str', 'month_name']).size().reset_index(name='count')
fig = px.density_heatmap(
    heatmap_data,
    x='month_name',
    y='hour_str',
    z='count',
    color_continuous_scale='Viridis',
    category_orders={"month_name": month_order},
    labels={'month_name': 'Month', 'hour_str': 'Hour', 'count': 'sightings'},
    nbinsx=12,
    nbinsy=24,
    height=800
)

bump_fonts(fig)
fig.show()

# The increase in sightings and reporting states shows the access to communication means has increased

In [ ]:
df['year'] = df['reported_date_time'].dt.year
df['duration_minutes'] = df['duration_seconds'].dt.total_seconds() / 60
state_year_counts = df.groupby(['state', 'year']).size().reset_index(name='sightings')
state_year_avg_duration = df.groupby(['state', 'year'])['duration_minutes'].mean().reset_index(name='avg_duration_minutes')

state_yearly = pd.merge(state_year_counts, state_year_avg_duration, on=['state', 'year'])

merged = pd.merge(state_yearly, population_df, on=['state'], how='left').dropna()
decades = list(range(1945, merged['year'].max() + 1, 5))
merged_decade = merged[merged['year'].isin(decades)].copy()
merged_decade = merged_decade.sort_values(['year', 'state'])

fig = px.scatter(
    merged_decade,
    x='avg_duration_minutes',
    y='sightings',
    color='state',
    size='population',
    facet_col='year',
    facet_col_wrap=4,
    labels={'sightings': 'Sightings Count', 'avg_duration_minutes': 'Average Duration [m]', 'state': 'State'},
    height=800
)
bump_fonts(fig)
fig.show()
